### Setup

In [2]:
import os
import os.path
import glob
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from datetime import datetime
from functools import reduce
import preprocess_data as prep
from caffe_classes import class_names

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Path to the project
project_path = '/Users/vladarozova/Dropbox/Bladder cancer/CAD/'

# Path to bladder cancer training data
data_path = os.path.join(project_path, 'images/')

# Path to bladder cancer test data
test_data_path = os.path.join(project_path, 'test set/')

# Path to labels for bladder cancer data
labels_path = os.path.join(project_path, 'training set/labels/')

# Path to masks for bladder cancer data
masks_path = os.path.join(project_path, 'training set/masks/')

# Path to ImageNet training data
train_on_imagenet = os.path.join(project_path, 'bladder_cancer_scripts/various models/imagenet/')

# Path to ImageNet test data
test_on_imagenet = os.path.join(project_path, 'bladder_cancer_scripts/various models/imagenet/')

# Path to weights of the pretrained VGG16 model
weights_name = os.path.join(project_path, 'weights/vgg16.npy')

# Store current directory
current_dir = os.getcwd()

# Input dimensions for VGG16
input_height = 224
input_width = 224

### Load bladder cancer data

In [3]:
# Load images and corresponding masks for UNet
images, masks = prep.load_images_with_masks(data_path, masks_path, input_height, input_width, binary=True)
assert len(images) == len(masks)

In [ ]:
# Load images and labels for whole-image binary classification
#images, labels = prep.load_images_with_labels(data_path, labels_path, input_height, input_width)
#assert images.shape[0] == labels.shape[0]

In [4]:
# Create stratified training and validation sets
X_train, y_train, X_val, y_val = prep.split_train_val(images, masks, train_ratio=0.8)

Training set: (100, 224, 224, 3) (100, 224, 224)
Validation set: (25, 224, 224, 3) (25, 224, 224)


### Build the model

In [5]:
def get_kernel(layer_name, trainable):
    """Returns kernel values for a conv layer from the pretrained model"""
    return tf.Variable(weights_dict[layer_name][0], name="kernel", trainable=trainable)

In [6]:
def get_bias(layer_name, trainable):
    """Returns bias values for a conv layer from the pretrained model"""
    return tf.Variable(weights_dict[layer_name][1], name="bias", trainable=trainable)

In [7]:
def get_fc_weights(layer_name, trainable):
    """Returns weights for a fc layer from the pretrained model""" 
    return tf.Variable(weights_dict[layer_name][0], name="kernel", trainable=trainable)

In [8]:
def get_weights(layer_name, idx, name, trainable):
    """Returns weights from the pretrained model""" 
    return tf.Variable(weights_dict[layer_name][idx], name=name, trainable=trainable)

In [9]:
def conv_layer(x, n_neurons, layer_name, kernel_size=[3, 3], padding='SAME', activation=tf.nn.relu):
    with tf.variable_scope(layer_name):  
        # Check if this is a new layer
        if layer_name in new_layers:                
            return tf.layers.conv2d(x, n_neurons, kernel_size=kernel_size, padding=padding, activation=activation)
            
        # Check if the layer is trained 
        trainable = False
        if layer_name in train_layers:
            trainable = True
            
        # Get kernel values from weights_dict for 'layer_name'
        kernel = get_kernel(layer_name, trainable)

        # Get bias values from weights_dict for 'layer_name'
        bias = get_bias(layer_name, trainable)

        # Convolve input with preinitialized kernel
        conv = tf.nn.conv2d(x, kernel, strides=[1, 1, 1, 1], padding=padding)

        # Add bias
        layer = tf.nn.bias_add(conv, bias)
        
        # Apply activation
        if activation is not None:
            return activation(layer)
        else:
            return layer

In [10]:
def transpConv_layer(x, n_neurons, layer_name, activation=tf.nn.relu):
    with tf.variable_scope(layer_name):  
        return tf.layers.conv2d_transpose(x, n_neurons, [3, 3], strides=(2, 2), padding='SAME', activation=activation)

In [11]:
def max_pool(x, layer_name):
    """Returns the feature map downsampled by 2x"""
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME', name=layer_name)

In [12]:
def fc_layer(x, n_neurons, layer_name, activation=tf.nn.relu):
    with tf.variable_scope(layer_name):
        # Check if this is a new layer
        if layer_name in new_layers:
            return tf.layers.dense(x, n_neurons, activation=activation)
        
        # Check if the layer is trained
        trainable=False
        if layer_name in train_layers:
            trainable=True
            
        # Get weights from weights_dict for 'layer_name'
        weights = get_fc_weights(layer_name, trainable)

        # Get bias values from weights_dict for 'layer_name'
        bias = get_bias(layer_name, trainable)

        # Flatten the input
        shape = x.get_shape()
        new_shape = reduce(lambda x, y: x*y, shape[1:])
        x_flat = tf.reshape(x, [-1, new_shape])

        # Multiply input by weights
        fc = tf.matmul(x_flat, weights)

        # Add bias
        layer = tf.nn.bias_add(fc, bias, name='layer')

        # Apply activation
        if activation is not None:
            return activation(layer)
        else:
            return layer

In [13]:
def vgg16_unet(x, n_classes, input_height, input_width):
    with tf.variable_scope("VGG16_UNet") as scope:
        # Reshape the images to feed to VGG16_UNet 
        x_input = tf.reshape(x, [-1, input_height, input_width, 3])
        
        # Block 1 consists of 2 convolutional layers followed by max pool
        with tf.variable_scope("Block_1") as scope:
            conv1_1 = conv_layer(x_input, 64, 'conv1_1')
            conv1_2 = conv_layer(conv1_1, 64, 'conv1_2')
            pool1 = max_pool(conv1_2, 'pool1')
            
            print("conv1_1 shape: {}".format(conv1_1.shape))
            print("conv1_2 shape: {}".format(conv1_2.shape))
            print("pool1 shape: {}".format(pool1.shape))
        
        # Block 2 consists of 2 convolutional layers followed by max pool
        with tf.variable_scope("Block_2") as scope:
            conv2_1 = conv_layer(pool1, 128, 'conv2_1')
            conv2_2 = conv_layer(conv2_1, 128, 'conv2_2')
            pool2 = max_pool(conv2_2, 'pool2')
            
            print("conv2_1 shape: {}".format(conv2_1.shape))
            print("conv2_2 shape: {}".format(conv2_2.shape))
            print("pool2 shape: {}".format(pool2.shape))
        
        # Block 3 consists of 3 convolutional layers followed by max pool
        with tf.variable_scope("Block_3") as scope:
            conv3_1 = conv_layer(pool2, 256, 'conv3_1')
            conv3_2 = conv_layer(conv3_1, 256, 'conv3_2')
            conv3_3 = conv_layer(conv3_2, 256, 'conv3_3')
            pool3 = max_pool(conv3_3, 'pool3')
            
            print("conv3_1 shape: {}".format(conv3_1.shape))
            print("conv3_2 shape: {}".format(conv3_2.shape))
            print("conv3_3 shape: {}".format(conv3_3.shape))
            print("pool3 shape: {}".format(pool3.shape))
            
        # Block 4 consists of 3 convolutional layers followed by max pool
        with tf.variable_scope("Block_4") as scope:
            conv4_1 = conv_layer(pool3, 512, 'conv4_1')
            conv4_2 = conv_layer(conv4_1, 512, 'conv4_2')
            conv4_3 = conv_layer(conv4_2, 512, 'conv4_3')
            pool4 = max_pool(conv4_3, 'pool4')
            
            print("conv4_1 shape: {}".format(conv4_1.shape))
            print("conv4_2 shape: {}".format(conv4_2.shape))
            print("conv4_3 shape: {}".format(conv4_3.shape))
            print("pool4 shape: {}".format(pool4.shape))
            
        # Block 5 consists of 3 convolutional layers followed by max pool
        with tf.variable_scope("Block_5") as scope:
            conv5_1 = conv_layer(pool4, 512, 'conv5_1')
            conv5_2 = conv_layer(conv5_1, 512, 'conv5_2')
            conv5_3 = conv_layer(conv5_2, 512, 'conv5_3')
            pool5 = max_pool(conv5_3, 'pool5')
            
            print("conv5_1 shape: {}".format(conv5_1.shape))
            print("conv5_2 shape: {}".format(conv5_2.shape))
            print("conv5_3 shape: {}".format(conv5_3.shape))
            print("pool5 shape: {}".format(pool5.shape))
            
        # One convolution at the bottom of the UNet
        with tf.variable_scope("Bottom") as scope:
            bottom_conv = conv_layer(pool5, 1024, 'bottom_conv')
            
            print("bottom_conv shape: {}".format(bottom_conv.shape))
        
        # Block 6 consists a transposed convolution and concatenation
        # followed by 3 convolutional layers
        with tf.variable_scope("Block_6") as scope:
            print("Block 6")
            transpConv6 = transpConv_layer(bottom_conv, 512, 'transpConv6')
            
            print("transpConv6 shape: {}".format(transpConv6.shape))
            print("conv5_3 shape: {}".format(conv5_3.shape))
            print("Concatenation...")
            
            concat6 = tf.concat([transpConv6, conv5_3], axis=3) 
            print("concat6 shape: {}".format(concat6.shape))
            
            conv6_1 = conv_layer(concat6, 512, 'conv6_1')
            conv6_2 = conv_layer(conv6_1, 512, 'conv6_2')
            conv6_3 = conv_layer(conv6_2, 512, 'conv6_3')
            
            print("conv6_1 shape: {}".format(conv6_1.shape))
            print("conv6_2 shape: {}".format(conv6_2.shape))
            print("conv6_3 shape: {}".format(conv6_3.shape))
            
        # Block 7 consists a transposed convolution and concatenation
        # followed by 3 convolutional layers
        with tf.variable_scope("Block_7") as scope:
            print("Block 7")
            transpConv7 = transpConv_layer(conv6_3, 512, 'transpConv6')
            
            print("transpConv7 shape: {}".format(transpConv7.shape))
            print("conv4_3 shape: {}".format(conv4_3.shape))
            print("Concatenation...")
            
            concat7 = tf.concat([transpConv7, conv4_3], axis=3) 
            print("concat7 shape: {}".format(concat7.shape))
            
            conv7_1 = conv_layer(concat7, 512, 'conv7_1')
            conv7_2 = conv_layer(conv7_1, 512, 'conv7_2')
            conv7_3 = conv_layer(conv7_2, 512, 'conv7_3')
            
            print("conv7_1 shape: {}".format(conv7_1.shape))
            print("conv7_2 shape: {}".format(conv7_2.shape))
            print("conv7_3 shape: {}".format(conv7_3.shape))
            
        # Block 8 consists a transposed convolution and concatenation
        # followed by 3 convolutional layers
        with tf.variable_scope("Block_8") as scope:
            print("Block 8")
            transpConv8 = transpConv_layer(conv7_3, 256, 'transpConv8')
            
            print("transpConv8 shape: {}".format(transpConv8.shape))
            print("conv3_3 shape: {}".format(conv3_3.shape))
            print("Concatenation...")
            
            concat8 = tf.concat([transpConv8, conv3_3], axis=3) 
            print("concat8 shape: {}".format(concat8.shape))
            
            conv8_1 = conv_layer(concat8, 256, 'conv8_1')
            conv8_2 = conv_layer(conv8_1, 256, 'conv8_2')
            conv8_3 = conv_layer(conv8_2, 256, 'conv8_3')
            
            print("conv8_1 shape: {}".format(conv8_1.shape))
            print("conv8_2 shape: {}".format(conv8_2.shape))
            print("conv8_3 shape: {}".format(conv8_3.shape))
            
        # Block 9 consists a transposed convolution and concatenation
        # followed by 2 convolutional layers
        with tf.variable_scope("Block_9") as scope:
            print("Block 9")
            transpConv9 = transpConv_layer(conv8_3, 128, 'transpConv9')
            
            print("transpConv9 shape: {}".format(transpConv9.shape))
            print("conv2_2 shape: {}".format(conv2_2.shape))
            print("Concatenation...")
            
            concat9 = tf.concat([transpConv9, conv2_2], axis=3) 
            print("concat9 shape: {}".format(concat9.shape))
            
            conv9_1 = conv_layer(concat9, 128, 'conv9_1')
            conv9_2 = conv_layer(conv9_1, 128, 'conv9_2')
            
            print("conv9_1 shape: {}".format(conv9_1.shape))
            print("conv9_2 shape: {}".format(conv9_2.shape))
            
        # Block 10 consists a transposed convolution and concatenation
        # followed by 1 convolutional layers
        with tf.variable_scope("Block_10") as scope:
            print("Block 10")
            transpConv10 = transpConv_layer(conv9_2, 64, 'transpConv10')
            
            print("transpConv10 shape: {}".format(transpConv10.shape))
            print("conv1_2 shape: {}".format(conv1_2.shape))
            print("Concatenation...")
            
            concat10 = tf.concat([transpConv10, conv1_2], axis=3) 
            print("concat10 shape: {}".format(concat10.shape))
            
            conv10_1 = conv_layer(concat10, 64, 'conv10_1')
            
            print("conv10_1 shape: {}".format(conv10_1.shape))
        
        # The last layer is a 1x1 convolution that maps each
        # 64-component feature vector to the desired number of classes
        with tf.variable_scope("Final_layer") as scope:
            print("Final layer")
            final = conv_layer(conv10_1, n_classes, 'final', kernel_size=[1, 1], padding='VALID', activation=None)
            
            print("final shape: {}".format(final.shape))
            
    return final

### Configuration settings

In [14]:
def reset_graph(seed=42):
    """
    This function resets the currrent TF graph
    to avoid duplicate nodes
    """
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

In [15]:
def log_dir(prefix=""):
    """
    This function creates a folder for TF logs
    """
    now = datetime.now().strftime("%H%M%S%d%m%Y")
    root_logdir = "tf_logs_unet"
    if prefix:
        prefix += "-"
    name = prefix + "run-" + now
    return "{}/{}/".format(root_logdir, name)

In [16]:
def batch_with_reps(X, y, batch_size):
    """
    This function extracts a new batch with repetiotions
    """
    rnd_ind = np.random.randint(0, len(X), batch_size)
    X_batch = X[rnd_ind, :, :, :]
    y_batch = y[rnd_ind]
    return X_batch, y_batch

In [17]:
def labels_to_onehot(y, n_classes):
    """
    This function converts labels to one-hot vectors
    """
    not_y = (y + 1) % 2

    one_hot = np.zeros((y.shape + (n_classes,)), dtype = np.float32)
    one_hot[:,:,:,0] = not_y
    one_hot[:,:,:,1] = y
    
    return one_hot

In [70]:
# Set current directory
os.chdir(current_dir)

# Reset graph
reset_graph()

# Learning params
learning_rate = 0.01
n_classes = 2
n_epochs = 3
batch_size = 1

# Network params
dropout_rate = 0.5

# Layers 
new_layers = ['bottom_conv', 
              'transpConv6', 'conv6_1', 'conv6_2', 'conv6_3', 
              'transpConv7', 'conv7_1', 'conv7_2', 'conv7_3', 
              'transpConv8', 'conv8_1', 'conv8_2', 'conv8_3', 
              'transpConv9', 'conv9_1', 'conv9_2', 
              'transpConv10', 'conv10_1', 'final']
train_layers = ''#new_layers + ['conv5_3'] +['conv5_2'] + ['conv5_1'] + ['conv4_3'] + ['conv4_2'] + ['conv4_1']
 
# Load weights
weights_dict = np.load(weights_name, encoding='latin1', allow_pickle = True).item()

# Path for tf.summary.FileWriter
filewriter_path = log_dir("VGG16_UNet")

# Path to store checkpoints
checkpoint_path = "/tmp/vgg16_unet/"
final_model_path = os.path.join(checkpoint_path, 'vgg16_unet_final_model.ckpt')

# Create parent path if it doesn't exist
if not os.path.isdir(checkpoint_path): 
    os.mkdir(checkpoint_path)

### Construction phase

In [68]:
def dice_loss(y_true, y_pred):
    numerator = 2 * tf.reduce_sum(y_true * y_pred, axis=-1)
    denominator = tf.reduce_sum(y_true + y_pred, axis=-1)
    dice = 1 - (numerator + 1) / (denominator + 1)
#     dice = tf.reduce_mean(dice)
    
    return dice

In [71]:
# TF placeholders for graph input and output
X = tf.placeholder(tf.float32, shape=[None, input_height, input_width, 3], name="X")
y = tf.placeholder(tf.float32, shape=[None, input_height, input_width, n_classes], name="y")

# Output of the model
logits = vgg16_unet(X, n_classes, input_height, input_width)
print("logits shape: {}".format(logits.shape))

# Apply sigmoid function to obtain probabilities
y_proba = tf.nn.softmax(logits, name="probabilities") # by default axis=-1
print("probabilities shape: {}".format(y_proba.shape))

# Round to obtain predictions
y_pred = tf.round(y_proba, name="predictions")
print("predictions shape: {}".format(y_pred.shape))

print("y shape: {}".format(y.shape))

# Assert the output has the same shape as the input
#assert logits.shape == y.shape

if train_layers:
    # List of trainable variables
    train_vars = [v for v in tf.trainable_variables() if v.name.split('/')[2] in train_layers]
    print("{} trainable variables:".format(len(train_vars)))
    for v in train_vars:
        print(v.name)

# Define the loss function
with tf.name_scope("loss"):
#     y_flat = tf.reshape(y, [-1, n_classes])
#     logits_flat = tf.reshape(logits, [-1, n_classes])
#     loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_flat, logits=logits_flat))

    y_flat = tf.reshape(y, [-1, 1])
    pred_flat = tf.reshape(y_pred, [-1, 1])
    loss = dice_loss(y_flat, pred_flat)

    # TODO: add smoothing term
    #smooth = tf.reduce_mean(tf.squared_difference(pred_maps[:, 0:height-1, :, :], pred_maps[:, 1:, :, :]))
    #smooth = tf.constant(weight, dtype=tf.float32)*(smooth + tf.reduce_mean(tf.squared_difference(pred_maps[:, :, 0:width-1, :], pred_maps[:, :, 1:, :])))
    #smoothed_loss = loss + smooth
    
# Define the training op
with tf.name_scope("train"):
    if train_layers:
        # Get gradients of variables that will be visualised in TensorBoard
        gradients = tf.gradients(loss, train_vars)
        gradients = list(zip(gradients, train_vars))
    
    # Create optimizer and apply Adam optimizer to the trainable variables
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    # By default gradients are computed using 
    # GraphKeys.TRAINABLE_VARIABLES collection
#     assert tf.trainable_variables() == train_vars
    training_op = optimizer.minimize(loss)
    
# Define performance metrics and operations
with tf.name_scope("eval"):
    # IoU
    labels = tf.cast(y[:, :, :, 1], tf.int32)
    predictions = tf.cast(y_pred[:, :, :, 1], tf.int32)
    iou, op = tf.metrics.mean_iou(labels, predictions, n_classes, name="iou")
    
    # Custom IoU from YPDL
    meanIoU = tf.Variable(0.0, trainable=False, dtype=tf.float32)
    classIoU = []
    pred_max = tf.reduce_max(y_proba, axis=3) #[, , ,]

    for c in range(n_classes):
        class_map = y_proba[:, :, :, c]
        class_max = tf.equal(class_map, pred_max) # bool
        gt_class = tf.cast(y[:, :, :, c], tf.bool)
        Inter = tf.reduce_sum(tf.cast(tf.logical_and(gt_class, class_max), tf.float32), axis=[1, 2])
        Union = tf.reduce_sum(tf.cast(tf.logical_or(gt_class, class_max), tf.float32), axis=[1, 2])
        # check for validity
        #tf.assert_greater_equal(Union, Inter)
        #print "Inter shape: {}".format(Inter.shape)
        #print "Union shape: {}".format(Union.shape)
        # add extra ones to union to prevent 0/0
        cIoU = Inter / (Union + tf.ones([batch_size], dtype=tf.float32))
        class_validNum = tf.greater_equal(tf.reduce_sum(y[:, :, :, c], axis=[1, 2]),
                                          tf.ones([batch_size], dtype=np.float32))
        class_validNum = tf.reduce_sum(tf.cast(class_validNum, tf.float32))
        cIoU = tf.reduce_sum(cIoU) / (tf.constant(0.01, dtype=tf.float32) + class_validNum)
        classIoU.append(cIoU)
        meanIoU = meanIoU + cIoU
    valid_classes = tf.greater_equal(tf.reduce_sum(y, axis=[0, 1, 2]), tf.ones([n_classes], dtype=np.float32))
    valid_classes = tf.reduce_sum(tf.cast(valid_classes, tf.float32))
    meanIoU = meanIoU / (tf.constant(0.01, dtype=tf.float32) + valid_classes)
    
    # Accuracy
    accuracy, op1 = tf.metrics.accuracy(tf.argmax(y, 1), tf.argmax(y_pred, 1), name="accuracy")
    
    # Precision
    precision, op2 = tf.metrics.precision(tf.argmax(y, 1), tf.argmax(y_pred, 1), name="precision")
    
    # Recall
    recall, op3 = tf.metrics.recall(tf.argmax(y, 1), tf.argmax(y_pred, 1), name="recall")
    
    # F-score
    fscore = tf.cast(2 * precision * recall / (precision + recall), tf.float32, name="fscore")
    
# Where to add summary
with tf.name_scope("summary") as scope: 

    # Add gradients to summary  
    for gradient, var in gradients:
        tf.summary.histogram(var.name + '/gradient', gradient)
        
    # Add the variables we train to the summary  
    for var in train_vars:
        tf.summary.histogram(var.name, var)

    # Add the loss to the  summary
    loss_summary = tf.summary.scalar('Cross_entropy', loss)
    
    # Add IoU to the summary
    iou_summary = tf.summary.scalar('IoU', iou)
    
    # Add custom IoU to the summary
    for c in range(n_classes):
        tf.summary.scalar('class{}_IoU'.format(c), classIoU[c])
    tf.summary.scalar('meanIoU', meanIoU)
      
    # Add the accuracy to the summary
    tf.summary.scalar('Accuracy', accuracy)
      
    # Add the fscore to the summary
    tf.summary.scalar('Fscore', fscore)

    # Merge all summaries together
    merged_summary = tf.summary.merge_all()
    
# Initialize the FileWriter
writer = tf.summary.FileWriter(filewriter_path)

# Global and local variables initializers
global_init = tf.global_variables_initializer()
local_init = tf.local_variables_initializer()

# Initialize a saver for store model checkpoints
saver = tf.train.Saver()

conv1_1 shape: (?, 224, 224, 64)
conv1_2 shape: (?, 224, 224, 64)
pool1 shape: (?, 112, 112, 64)
conv2_1 shape: (?, 112, 112, 128)
conv2_2 shape: (?, 112, 112, 128)
pool2 shape: (?, 56, 56, 128)
conv3_1 shape: (?, 56, 56, 256)
conv3_2 shape: (?, 56, 56, 256)
conv3_3 shape: (?, 56, 56, 256)
pool3 shape: (?, 28, 28, 256)
conv4_1 shape: (?, 28, 28, 512)
conv4_2 shape: (?, 28, 28, 512)
conv4_3 shape: (?, 28, 28, 512)
pool4 shape: (?, 14, 14, 512)
conv5_1 shape: (?, 14, 14, 512)
conv5_2 shape: (?, 14, 14, 512)
conv5_3 shape: (?, 14, 14, 512)
pool5 shape: (?, 7, 7, 512)
bottom_conv shape: (?, 7, 7, 1024)
Block 6
transpConv6 shape: (?, 14, 14, 512)
conv5_3 shape: (?, 14, 14, 512)
Concatenation...
concat6 shape: (?, 14, 14, 1024)
conv6_1 shape: (?, 14, 14, 512)
conv6_2 shape: (?, 14, 14, 512)
conv6_3 shape: (?, 14, 14, 512)
Block 7
transpConv7 shape: (?, 28, 28, 512)
conv4_3 shape: (?, 28, 28, 512)
Concatenation...
concat7 shape: (?, 28, 28, 1024)
conv7_1 shape: (?, 28, 28, 512)
conv7_2 shape:

ValueError: No gradients provided for any variable, check your graph for ops that do not support gradients, between variables ["<tf.Variable 'VGG16_UNet/Bottom/bottom_conv/conv2d/kernel:0' shape=(3, 3, 512, 1024) dtype=float32_ref>", "<tf.Variable 'VGG16_UNet/Bottom/bottom_conv/conv2d/bias:0' shape=(1024,) dtype=float32_ref>", "<tf.Variable 'VGG16_UNet/Block_6/transpConv6/conv2d_transpose/kernel:0' shape=(3, 3, 512, 1024) dtype=float32_ref>", "<tf.Variable 'VGG16_UNet/Block_6/transpConv6/conv2d_transpose/bias:0' shape=(512,) dtype=float32_ref>", "<tf.Variable 'VGG16_UNet/Block_6/conv6_1/conv2d/kernel:0' shape=(3, 3, 1024, 512) dtype=float32_ref>", "<tf.Variable 'VGG16_UNet/Block_6/conv6_1/conv2d/bias:0' shape=(512,) dtype=float32_ref>", "<tf.Variable 'VGG16_UNet/Block_6/conv6_2/conv2d/kernel:0' shape=(3, 3, 512, 512) dtype=float32_ref>", "<tf.Variable 'VGG16_UNet/Block_6/conv6_2/conv2d/bias:0' shape=(512,) dtype=float32_ref>", "<tf.Variable 'VGG16_UNet/Block_6/conv6_3/conv2d/kernel:0' 

In [ ]:
filewriter_path

### Checking graph, operations, variables

In [ ]:
# Start Tensorflow session
with tf.Session() as sess:
 
    # Initialize all variables
    sess.run([global_init, local_init])
  
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
writer.close()    

In [ ]:
for op in tf.get_default_graph().get_operations():
    print(op.values())

In [ ]:
for var in tf.trainable_variables():
    print(var.name)

In [ ]:
for var in check_vars:
    print(var.name, var)

### Load data

In [62]:
# Create one-hot labels
y_train_onehot = labels_to_onehot(y_train, n_classes)
y_train_onehot.shape

(100, 224, 224, 2)

In [63]:
# Get the number of training/validation steps per epoch
train_batches_per_epoch = np.floor(len(X_train) / batch_size).astype(np.int16)
val_batches_per_epoch = np.floor(len(X_val) / batch_size).astype(np.int16)
print(train_batches_per_epoch)
print(val_batches_per_epoch)

100
25


In [64]:
# Start Tensorflow session
with tf.Session() as sess:
 
    # Initialize all variables
    sess.run([global_init, local_init])
  
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
  
    print("{} Start training...".format(datetime.now()))
    print("{} Open Tensorboard at --logdir {}".format(datetime.now(), filewriter_path))
  
    # Loop over number of epochs
    for epoch in range(n_epochs):    
        print("{} Epoch number: {}".format(datetime.now(), epoch + 1))
        
        for batch_index in range(train_batches_per_epoch):
            # Get a new batch of images and labels
            X_batch, y_batch = batch_with_reps(X_train, y_train_onehot, batch_size)
            feed_dict = {X: X_batch, y: y_batch}
            
            # Run the training op
            sess.run(training_op, feed_dict=feed_dict)
            
            # Check the performance every 10 batches
            if batch_index % 10 == 0:
                feed_dict={X: X_batch, y: y_batch}
                step = epoch * train_batches_per_epoch + batch_index
                
                # Update ops to correctly calculate tf_metrics
                sess.run([op, op1, op2, op3], feed_dict=feed_dict)
                
                # Calculate tf_metrics and write the summary
                summary_str = sess.run(merged_summary, feed_dict=feed_dict)
                writer.add_summary(summary_str, step)
                
                # Flush the event file to disk
                writer.flush()
        """
        # Validate the model on the validation set using batches after each epoch
        print("{} Start validation".format(datetime.now()))
        val_acc = 0.
        val_count = 0
        for _ in range(val_batches_per_epoch):
            X_batch, y_batch = batch_with_reps(X_val_std, y_val_onehot, batch_size)
            acc = sess.run(accuracy, feed_dict={X: X_batch, y: y_batch, keep_prob: 1.})
            val_acc += acc
            val_count += 1
        val_acc /= val_count
        print("{} Validation Accuracy = {:.4f}".format(datetime.now(), val_acc))
        """
        # Save checkpoint of the model
        print("{} Saving checkpoint of model...".format(datetime.now()))  
        
        #checkpoint_name = os.path.join(checkpoint_path, 'model_epoch'+str(epoch+1)+'.ckpt')
        #saver.save(sess, checkpoint_name)  
        
        #print("{} Model checkpoint saved at {}".format(datetime.now(), checkpoint_name))
    
    # Save the final model
    print("{} Saving the final model...".format(datetime.now()))
    
    #saver.save(sess, final_model_path)
    
    print("{} Final model saved at {}".format(datetime.now(), final_model_path))
    
    # Isolate the variables stored behind the scenes by the metric operation
    #stream_vars = [i for i in tf.local_variables() if 'fscore' in i.name] # variables associated with F score
    
    # Initialize/reset the selected running variables
    #stream_vars_init = tf.variables_initializer(var_list=stream_vars)
    #sess.run(stream_vars_init)
    
    # Validate the final model on the entire validation set
    proba_val, fscore_val, iou_val = sess.run([y_proba, fscore, iou], feed_dict={X: X_val})
    
writer.close()

2019-11-08 07:25:27.580141 Start training...
2019-11-08 07:25:27.816320 Open Tensorboard at --logdir tf_logs_unet/VGG16_UNet-run-07251208112019/
2019-11-08 07:25:27.816357 Epoch number: 1
2019-11-08 07:29:27.614551 Saving checkpoint of model...
2019-11-08 07:29:27.614736 Epoch number: 2
2019-11-08 07:33:31.591807 Saving checkpoint of model...
2019-11-08 07:33:31.591956 Epoch number: 3
2019-11-08 07:37:21.566085 Saving checkpoint of model...
2019-11-08 07:37:21.566218 Saving the final model...
2019-11-08 07:37:21.566242 Final model saved at /tmp/vgg16_unet/vgg16_unet_final_model.ckpt


In [ ]:
# print(np.sum(proba_val[:,:,:,0]==0.7339438))
# print(25*224*224)
# proba_val[proba_val[:,:,:,0]!=0.7339438]

### Validation

In [ ]:
proba_val

In [ ]:
pred = (proba_val[:,1] >= 0.5)
print("Total:", pred.size)
print("Condition positive", y_val.sum())
print("Predicted positive:", pred.sum())

In [ ]:
from sklearn.metrics import precision_score, recall_score

print("Precision:", precision_score(y_val, pred))
print("Recall:", recall_score(y_val, pred))

In [ ]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_val, pred, labels = [0, 1])

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_val, pred) 

In [ ]:
from sklearn.metrics import roc_curve

fpr, tpr, threshold = roc_curve(y_val, proba[:,1])

def plot_roc_curve(fpr, tpr, label=None):
    plt.plot(fpr, tpr, linewidth=2, label=label)
    plt.plot([0, 1], [0, 1], 'k--')
    plt.axis([0, 1, 0, 1])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Posiitve Rate')
    
plot_roc_curve(fpr, tpr)
plt.show()

In [ ]:
from sklearn.metrics import precision_recall_curve

prec, rec, threshold = precision_recall_curve(y_val, proba[:,1])

def plot_roc_curve(prec, rec, label=None):
    plt.plot(rec, prec, linewidth=2, label=label)
    plt.axis([0, 1, 0, 1])
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    
plot_roc_curve(prec, rec)
plt.show()

### Load test images from ImageNet 

In [ ]:
from caffe_classes import class_names

In [ ]:
test_images = prep.load_imagenet_images(test_on_imagenet, input_height, input_width)

### Test the pretrained model on ImageNet 

In [ ]:
with tf.Session() as sess:
    
    # Initialize all variables
    sess.run(tf.global_variables_initializer())
    
    # Add the model graph to TensorBoard
    writer.add_graph(sess.graph)
    
    # Create figure handle
    fig2 = plt.figure(figsize=(15,6))
    
    # Loop over all images
    for i, image in enumerate(test_images):
        
        img = image.reshape((1, input_height, input_width, 3))
        
        # Run the session and calculate the class probability
        proba = sess.run(y_proba, feed_dict={X: img})
        
        # Get the class name of the class with the highest probability
        class_name = class_names[np.argmax(proba)]
        
        # Plot image with class name and prob in the title
        #fig2.add_subplot(1,3,i+1)
        plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        plt.title("Class: " + class_name + ", probability: %.4f" %proba[0,np.argmax(proba)])
        plt.axis('off')
        
writer.close()